In [1]:
import sys
sys.path.append('..')

In [2]:
from core.dataset import *
from core.dprocessing import *
from core.optimizers import *
from core.utils import *
from core.objects import *
import seaborn as sns
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

In [3]:
load_dir = 'D:/Data/hoffmanlab/featureselection/data/'
ligands = ['CpG', 'FLA', 'FSL', 'LPS', 'P3K', 'PIC', 'R84', 'TNF']
sheet_type = 'am'

In [4]:
df = pd.read_csv('C:/Users/minha/Research/hoffmanlab/featuresel/data/untuned_model_feature_importance.csv')
feature_importance = df.values.tolist()

In [5]:
dataset = Data(load_dir, ligands, sheet_type, merge=True, numpy=False)
labels = dataset.iloc[:, [984]]

In [6]:
features1 = ['integrals_pos', 'integrals', 'time_series', 'envelope', 'fold_change', 'valley_amps', 'derivatives']
timestep1 = [98, 98, 98, 25, 98, 15, 96] 

## testing new optimizer object for efficiency etc

In [ ]:
model = xgb.XGBClassifier(tree_method='gpu_hist', use_label_encoder=False)

In [ ]:
optim = Optimizer(model, features1, timestep1, feature_importance, labels, dataset, ligands)

In [ ]:
optim.model

In [ ]:
optim.fit(bound=0.4)

In [ ]:
optim.model

In [ ]:
new_feature_importances = optim.model.feature_importances_.argsort()

In [ ]:
new_feature_importances = new_feature_importances[::-1]

In [ ]:
nfi_names = optim.df.columns[new_feature_importances]
# need to turn into list of lists bc functions are kinda trash rn

In [ ]:
nfi_names = pd.DataFrame(nfi_names)

In [ ]:
model2 = xgb.XGBClassifier(tree_method='gpu_hist', use_label_encoder=False)

In [ ]:
optim2 = Optimizer(model2, features1, timestep1, nfi_names.values, labels, dataset, ligands)

In [ ]:
optim2.fit(bound=0.75) # bound does not necessarily have to be 0.4

### gridsearching now

#### below parameters are just some params i found online, relatively arbitrary (i think)

In [16]:
gridsearchmodel = xgb.XGBClassifier(tree_method='gpu_hist', use_label_encoder=False, learning_rate=0.1, n_estimators=1000)

In [ ]:
optim3 = Optimizer(gridsearchmodel, features1, timestep1, feature_importance, labels, dataset, ligands)

In [ ]:
optim3.fit(bound=0.4)

In [ ]:
# slight improvments with these params, need to gridsearch tune

In [7]:
data = dataset.iloc[:, [i for i in range(984)]]

In [8]:
labels = dataset.iloc[:, [984]]

In [9]:
datan = data.to_numpy()
labelsn = labels.to_numpy()

In [10]:
labelsn = labelsn.reshape((-1, ))

In [11]:
partitioned_data = partition_features(features1, timestep1, feature_importance, 0.4, dataset)

In [12]:
X = partitioned_data.to_numpy()
Y = labelsn

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(datan, labelsn, test_size=0.1, random_state=42)

In [13]:
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }

In [14]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [17]:
folds = 3
param_comb = 5

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 42)
random_search = RandomizedSearchCV(gridsearchmodel, param_distributions=params, n_iter=param_comb, n_jobs=4, cv=skf.split(X, Y), 
                                   verbose=3, random_state=42)

In [ ]:
# random_search = GridSearchCV(estimator=gridsearchmodel, param_grid=params, scoring='roc_auc', n_jobs=4, cv=skf.split(X, Y), verbose=1)

### Brute force gridsearch

In [18]:
random_search.fit(X, Y)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


C:\Users\minha\miniconda3\envs\peep\lib\site-packages\sklearn\model_selection\_search.py:969: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan]
  warnings.warn(


[19:57:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


RandomizedSearchCV(cv=<generator object _BaseKFold.split at 0x00000208AFDA0DD0>,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           enable_categorical=False, gamma=None,
                                           gpu_id=None, importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate=0.1,
                                           max_delta_step=None, max_depth=None,
                                           min...
                                           reg_alpha=None, reg_lambda=None,
                                           scale_pos_weight=None,
                                           subsample=None,
                     

In [19]:
random_search.best_params_

{'subsample': 0.8,
 'min_child_weight': 10,
 'max_depth': 4,
 'gamma': 1.5,
 'colsample_bytree': 0.6}

In [20]:
gridsearchmodel

XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None,
              enable_categorical=False, gamma=None, gpu_id=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_delta_step=None, max_depth=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=1000, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=None, subsample=None,
              tree_method='gpu_hist', use_label_encoder=False,
              validate_parameters=None, verbosity=None)